# Output outcomes

depends on download_sheets

- tables_index.csv
- tables(tables/*.csv)
- l1.csv, l2.csv, l3.csv, l4.csv

In [ ]:
# Output table_index and tables for outcomes

import re
import os
import pandas as pd
from lib.utils import OUTPUT_OUTCOMES_DIR,OUTPUT_OUTCOMES_TABLE_DIR
from lib.apply_condition_to_dataframe  import apply_condition_to_dataframe
from lib.outcomes_utils import get_table_index,iter_tables_for_outcome_raw

os.makedirs(OUTPUT_OUTCOMES_DIR,exist_ok=True)
os.makedirs(OUTPUT_OUTCOMES_TABLE_DIR,exist_ok=True)

for table,info in iter_tables_for_outcome_raw():
    table = table\
        .loc[:,[*re.split(r" *, *",info.列),"index","UID"] ]
    print(f"output... {info.id}.csv")
    table.to_csv(f"{OUTPUT_OUTCOMES_TABLE_DIR}/{info.id}.csv",index=False)

table_index = get_table_index().drop("データ元",axis=1)
table_index.to_csv(f"{OUTPUT_OUTCOMES_DIR}/tables_index.csv",encoding="utf_8_sig",index=False)
print(f"output... tables_index.csv")


In [ ]:
# Output outcomes l1,l2,l3,l4

import csv
import os
import pandas as pd
import re
import glob
from lib.utils import BASE_DIR,SHEETS_OUTCOMES_DIR,OUTPUT_OUTCOMES_DIR,get_glob_file
from lib.dataframe_to_grouped_numbers import dataframe_to_grouped_numbers
from lib.outcomes_utils import get_table_index,format_table_ref

os.makedirs(f"{OUTPUT_OUTCOMES_DIR}",exist_ok=True)


# reading layer 1
r4_l1=pd.read_csv(f"{SHEETS_OUTCOMES_DIR}/第1層/第1層.csv")
r4_l1=r4_l1.rename(columns={"第1層イニシャル":"index","第1層フルスペル":"l1_spell","第1層":"l1","第1層説明":"l1_desc"})
r4_l1=r4_l1.loc[:,["index","UID","l1_spell","l1","l1_desc"]]
r4_l1.to_csv(f"{OUTPUT_OUTCOMES_DIR}/l1.csv",encoding="utf_8_sig",quoting=csv.QUOTE_NONNUMERIC,index=False)
print("output... l1.csv")


l1_indexes = r4_l1["index"]
# reading layer 2
r4_l2 =  pd.DataFrame(data=[],columns=[])
for i, row in r4_l1.iterrows():
    l1_index = row["index"]
    l1_UID = row["UID"]
    filename = get_glob_file(f"{SHEETS_OUTCOMES_DIR}/{l1_index}*/第2層.csv")
    r4_l2_unit =pd.read_csv(filename,encoding="utf_8_sig") 
    r4_l2_unit=r4_l2_unit.rename(columns={"第2層":"l2","第2層説明":"l2_desc"})
    r4_l2_unit["index"] = l1_index+"-"+(r4_l2_unit.index+1).astype("str").str.zfill(2)
    r4_l2_unit["l1_index"] = l1_index
    r4_l2_unit["l1_UID"] = l1_UID
    r4_l2_unit = r4_l2_unit.loc[:,["index","UID","l2","l2_desc","l1_index","l1_UID"]]
    r4_l2=pd.concat([r4_l2,r4_l2_unit])
r4_l2.to_csv(f"{OUTPUT_OUTCOMES_DIR}/l2.csv",encoding="utf_8_sig",quoting=csv.QUOTE_NONNUMERIC,index=False)
print("output... l2.csv")

# reading layer 3 and 4
r4_l3 =  pd.DataFrame(data=[],columns=[])
r4_l4 =  pd.DataFrame(data=[],columns=[])
for l1_index in l1_indexes:
    filename = get_glob_file(f"{SHEETS_OUTCOMES_DIR}/{l1_index}*/第2から4層.csv")
    r4_l234_unit=pd.read_csv(filename)
    r4_l234_unit=r4_l234_unit.rename(columns={"第2層":"l2","第3層":"l3","第4層":"l4","H28対応項目":"H28ID"})
    r4_l234_unit = pd.merge(r4_l234_unit,r4_l2.rename(columns={"UID":"l2_UID","index":"l2_index"}),how="left",on="l2")
    ids=dataframe_to_grouped_numbers(r4_l234_unit,["l2","l3","l4"])

    # reading layer 3
    r4_l234_unit["l3_index"]=r4_l234_unit["l2_index"]+"-"+ids["l3"].astype("str").str.zfill(2)
    r4_l3_unit = r4_l234_unit.loc[:,["l3_index","l3_UID","l3","l2_index","l2_UID"]]
    r4_l3_unit = r4_l3_unit.rename(columns={"l3_index":"index","l3_UID":"UID"})
    r4_l3 = pd.concat([r4_l3,r4_l3_unit.drop_duplicates(subset=["index"])]) 

    # reading layer 4
    r4_l234_unit["l4_index"]=r4_l234_unit["l3_index"]+"-"+ids["l4"].astype("str").str.zfill(2)
    r4_l4_unit = r4_l234_unit.loc[:,["l4_index","UID","l4","l3_index","l3_UID","H28ID"]]
    r4_l4_unit = r4_l4_unit.rename(columns={"l4_index":"index"})
    r4_l4 = pd.concat([r4_l4,r4_l4_unit]) 


r4_l4["l4"] = r4_l4["l4"].map(format_table_ref)
r4_l3["l3"] = r4_l3["l3"].map(format_table_ref)


r4_l3.to_csv(f"{OUTPUT_OUTCOMES_DIR}/l3.csv",encoding="utf_8_sig",quoting=csv.QUOTE_NONNUMERIC,index=False)
print("output... l3.csv")


r4_l4.to_csv(f"{OUTPUT_OUTCOMES_DIR}/l4.csv",encoding="utf_8_sig",quoting=csv.QUOTE_NONNUMERIC,index=False)
print("output... l4.csv")

